In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import json
import pandas as pd

In [ ]:
BASE_URL = "https://www.idrd.gov.co"
START_URL = "https://www.idrd.gov.co/nuestros-programas"

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; DataCollector/1.0; +https://yourdomain.example)"
}

resp = requests.get(START_URL, headers=headers, timeout=15)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

cards = []
# Buscar cualquier string que contenga "Escenario:" y tomar su contenedor más cercano
for marker in soup.find_all(string=re.compile(r'Escenario:', re.IGNORECASE)):
    # subir hasta un contenedor relevante (p.ej. el div padre con varias líneas)
    container = marker.find_parent()
    # intentar encontrar el contenedor que tiene todos los campos (subir más si es necesario)
    for _ in range(4):
        # heurística: si el contenedor contiene "Horario" y "Dirección", lo asumimos como tarjeta
        text = container.get_text(separator="\n", strip=True)
        if re.search(r'Horario[:\s]', text) and re.search(r'Direcci[oó]n[:\s]', text):
            break
        if container.parent:
            container = container.parent

    text = container.get_text(separator="\n", strip=True)
    # Extraer campos por líneas (hecha una extracción robusta por regex)
    datos = {}
    datos['raw_text'] = text

    # Title: primera línea antes de los asteriscos o separadores (heurística)
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    # buscar título: normalmente la primera línea que no es "Escenario" ni "Horario"
    titulo = None
    for line in lines[:3]:
        if not re.search(r'Escenario:|Direcci[oó]n:|Horario:|D[ií]as:|Localidad:', line, re.IGNORECASE):
            titulo = line
            break
    datos['titulo'] = titulo or lines[0] if lines else None

    # Extraer campos con regex
    def extraer(pattern, text):
        m = re.search(pattern, text, re.IGNORECASE)
        return m.group(1).strip() if m else None

    datos['escenario'] = extraer(r'Escenario[:\s]*([^\n\r]+)', text)
    datos['direccion'] = extraer(r'Direcci[oó]n[:\s]*([^\n\r]+)', text)
    datos['horario'] = extraer(r'Horario[:\s]*([^\n\r]+)', text)
    datos['dias'] = extraer(r'D[ií]as[:\s]*([^\n\r]+)', text)
    datos['localidad'] = extraer(r'Localidad[:\s]*([^\n\r]+)', text)

    # enlace "Más información" relativo al container
    a_mas = container.find("a", string=re.compile(r'Más información', re.IGNORECASE))
    if a_mas and a_mas.get("href"):
        datos['mas_info_url'] = urljoin(BASE_URL, a_mas['href'])
    else:
        # buscar enlace cerca (en el siguiente sibling)
        a = container.find_next("a", string=re.compile(r'Más información', re.IGNORECASE))
        datos['mas_info_url'] = urljoin(BASE_URL, a['href']) if a and a.get("href") else None

    cards.append(datos)

# deduplicar por (titulo, escenario)
unique = {}
for c in cards:
    key = (c.get('titulo'), c.get('escenario'))
    unique[key] = c
result = list(unique.values())

print(json.dumps(result, ensure_ascii=False, indent=2))

[
  {
    "raw_text": "Persona Mayor\nEscenario:\nPARQUE ESTADIO OLAYA HERRERA\nDirección:\nCARRERA 21 NUMERO 25-35 SUR\nHorario:\n7:00:00 a. m.\nDías:\nLUNES - VIERNES\nLocalidad:\nRafael Uribe Uribe\n* Fechas y horarios sujetos a cambios sin previo aviso",
    "titulo": "Persona Mayor",
    "escenario": "PARQUE ESTADIO OLAYA HERRERA",
    "direccion": "CARRERA 21 NUMERO 25-35 SUR",
    "horario": "7:00:00 a. m.",
    "dias": "LUNES - VIERNES",
    "localidad": "Rafael Uribe Uribe",
    "mas_info_url": "https://www.idrd.gov.co/recreacion/actividad-fisica-y-deporte/persona-mayor"
  },
  {
    "raw_text": "Manzanas del cuidado\nEscenario:\nINTEGRACIÓN SOCIAL EL CAMINO( MANZANA ENGATIVÁ)\nDirección:\nCARRERA 69 47 43\nHorario:\n9:00:00 a. m.\nDías:\nLUNES - MARTES - MIÉRCOLES - JUEVES - VIERNES\nLocalidad:\nEngativá\n* Fechas y horarios sujetos a cambios sin previo aviso",
    "titulo": "Manzanas del cuidado",
    "escenario": "INTEGRACIÓN SOCIAL EL CAMINO( MANZANA ENGATIVÁ)",
    "direc

In [ ]:
df = pd.DataFrame(result)

# Guardar en CSV
df.to_csv("programas_idrd.csv", index=False, encoding="utf-8-sig")

print("Archivo CSV guardado como 'programas_idrd.csv'")

Archivo CSV guardado como 'programas_idrd.csv'
